In [1]:
pwd

'/Users/kelly/metis_v3/Project_5'

In [2]:
from pymongo import MongoClient
client = MongoClient()
db = client.goodreads
books = db.books

In [3]:
books.count()

10000

In [4]:
books.find_one()

{'_id': ObjectId('5b88855b21c3f05eeaf945eb'),
 'book_id': 1,
 'goodreads_book_id': 2767052,
 'best_book_id': 2767052,
 'work_id': 2792775,
 'books_count': 272,
 'isbn': 439023483,
 'isbn13': 9780439023480.0,
 'authors': 'Suzanne Collins',
 'original_publication_year': 2008.0,
 'original_title': 'The Hunger Games',
 'title': 'The Hunger Games (The Hunger Games, #1)',
 'language_code': 'eng',
 'average_rating': 4.34,
 'ratings_count': 4780653,
 'work_ratings_count': 4942365,
 'work_text_reviews_count': 155254,
 'ratings_1': 66715,
 'ratings_2': 127936,
 'ratings_3': 560092,
 'ratings_4': 1481305,
 'ratings_5': 2706317,
 'image_url': 'https://images.gr-assets.com/books/1447303603m/2767052.jpg',
 'small_image_url': 'https://images.gr-assets.com/books/1447303603s/2767052.jpg'}

In [5]:
ratings = db.ratings
booktags = db.booktags
tags = db.tags

In [6]:
#pip install mongojoin

In [7]:
import pprint

In [8]:
for book in db.books.aggregate([{'$sample':{'size':5}},
                    {'$project':{'_id':0,'Book ID':'$book_id','Title':'$original_title'}}
                   ]):
        pprint.pprint(book)

{'Book ID': 8210, 'Title': 'Den lille havfrue'}
{'Book ID': 7785, 'Title': 'Dawn'}
{'Book ID': 2406, 'Title': 'The Mammoth Hunters'}
{'Book ID': 7306, 'Title': 'Say Cheese and Die!'}
{'Book ID': 815, 'Title': 'No Country for Old Men'}


In [9]:
for book in db.books.aggregate([{'$sample':{'size':5}},
                    {'$lookup': {
                        'from': 'ratings',
                        'localField': 'book_id',
                        'foreignField': 'book_id',
                        'as': 'joined'},
                    },
                    {'$project':{'_id':0,'Book ID':'$book_id','Title':'$original_title'}}
                   ]):
        pprint.pprint(book)

{'Book ID': 4914, 'Title': 'Starless Night (Legacy of the Drow #2)'}
{'Book ID': 4629, 'Title': 'The Expats'}
{'Book ID': 5304, 'Title': 'Midnight'}
{'Book ID': 1180, 'Title': 'The Forty Rules of Love'}
{'Book ID': 8216, 'Title': "Hornet's Nest"}


In [10]:
for book in db.books.find({"book_id":2}):
    print('Book ID: ', book['book_id'])
    print('Title: ', book['original_title'])

Book ID:  2
Title:  Harry Potter and the Philosopher's Stone


In [11]:
def find_book_info(ID):
    for book in db.books.find({'book_id':ID}):
        print (book)

In [12]:
find_book_info(2)

{'_id': ObjectId('5b88855b21c3f05eeaf945ec'), 'book_id': 2, 'goodreads_book_id': 3, 'best_book_id': 3, 'work_id': 4640799, 'books_count': 491, 'isbn': 439554934, 'isbn13': 9780439554930.0, 'authors': 'J.K. Rowling, Mary GrandPré', 'original_publication_year': 1997.0, 'original_title': "Harry Potter and the Philosopher's Stone", 'title': "Harry Potter and the Sorcerer's Stone (Harry Potter, #1)", 'language_code': 'eng', 'average_rating': 4.44, 'ratings_count': 4602479, 'work_ratings_count': 4800065, 'work_text_reviews_count': 75867, 'ratings_1': 75504, 'ratings_2': 101676, 'ratings_3': 455024, 'ratings_4': 1156318, 'ratings_5': 3011543, 'image_url': 'https://images.gr-assets.com/books/1474154022m/3.jpg', 'small_image_url': 'https://images.gr-assets.com/books/1474154022s/3.jpg'}


In [13]:
def find_book_rating(ID):
    for book in db.books.find({'book_id':ID}):
        print (book['average_rating'])

In [14]:
def find_book_title(ID):
    for book in db.books.find({'book_id':ID}):
#        print('Book ID: ', book['book_id'])
        print(book['original_title'])

In [15]:
find_book_title(2)

Harry Potter and the Philosopher's Stone


In [16]:
ratings.find_one()['book_id'],ratings.find_one()['rating']

(258, 5)

In [22]:
#rating_cursor = ratings.find()
rating_cursor = ratings.aggregate([{'$sample':{'size':5}}])                       

In [23]:
from collections import defaultdict
rating_dict = defaultdict(list)
#rating_dict = {}

for rating in rating_cursor:
    print(rating['user_id'],rating['book_id'], rating['rating'])
 #   if rating['book_id'] not in rating_dict:
  #      rating_dict[str(rating['book_id'])] = rating['rating']
  #  else:
  #      rating_dict[str(rating['book_id'])].append(rating['rating'])
    
#    print (type(rating_dict[rating['book_id']]))

30510 39 5
32581 1410 5
203 955 4
29074 129 5
27385 6833 5


In [24]:
for k, v in rating_dict.items():
    print ('Keys: ', k, 'Ratings: ', v)

In [25]:
from scipy.sparse import coo_matrix

In [26]:
from scipy.sparse.linalg import svds

In [27]:
import numpy as np

In [28]:
#rating_cursor = ratings.find()
rating_cursor = ratings.aggregate([{'$sample':{'size':5}}])   

In [29]:
user_list = np.array([])
book_list = np.array([])
rating_list = np.array([])

for rating in rating_cursor:
    user_list = np.append(user_list,(rating['user_id']))
    book_list = np.append(book_list,(float(rating['book_id'])))
    rating_list = np.append(rating_list,(float(rating['rating'])))

In [30]:
user_list

array([16054.,  5827.,  2600.,   956., 43324.])

In [31]:
rating_list

array([4., 4., 5., 4., 4.])

In [33]:
book_list

array([9366.,  906., 8904., 6764.,  557.])

In [34]:
user_list.max()

43324.0

In [35]:
mat = coo_matrix((rating_list, (user_list-1, book_list-1)), shape=(int(user_list.max()), int(max(book_list))))

In [36]:
mat.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [37]:
mat.shape

(43324, 9366)

In [38]:
U, s, VT = svds(mat,k=100)

In [39]:
VT.shape

(100, 9366)

In [40]:
U.shape

(43324, 100)

In [483]:
s.shape

(100,)

In [484]:
s

array([2., 4., 4., 4., 5., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [485]:
VT.shape[1]

6907

Find Similar Items

In [486]:
def find_similar_books(itemID, VT, num_recom):
    recs = []
    for item in range(VT.T.shape[0]):
        if item != itemID:
            recs.append([item+1,np.dot(VT.T[itemID-1],VT.T[item])])
    final_rec = [(i[0],i[1]) for i in sorted(recs,key=lambda x: x[1],reverse=True)]
    return final_rec[:num_recom]

In [487]:
book_num = 1
num_recom = 20
for book in find_similar_books(book_num,VT,num_recom):
    print(book)
#    print(movies[movies.movieId == str(item[0])].Title,'\n')

(1, 0.016035944827905747)
(6658, 0.005616350295360898)
(37, 0.005546637268570494)
(1147, 0.005446302001750676)
(852, 0.005281443069964975)
(6310, 0.004863842026131715)
(1329, 0.004862747290287774)
(1298, 0.004802999138335555)
(2967, 0.004754692149351942)
(6043, 0.004610055241736656)
(5936, 0.004582862015159991)
(1212, 0.004535096812213384)
(2173, 0.004494139138826673)
(1328, 0.004458432004053792)
(4237, 0.0044544749627672695)
(1761, 0.004436051408114306)
(388, 0.0043717544657749805)
(2743, 0.004342899153924018)
(4131, 0.004278793919749392)
(3230, 0.004264403297756552)


In [488]:
def item_similarity(item1,item2):
    return np.dot(VT.T[item1],VT.T[item2])

In [489]:
item_similarity(2,18)

9.775309546633202e-05

Get Recommendations for Users

In [490]:
def get_recommends_for_user(userID, U, VT, num_recom):
    recs = []
    for item in range(VT.T.shape[0]):
        recs.append([item+1,np.dot(U[userID-1],VT.T[item])])
    final_rec = [(i[0],i[1]) for i in sorted(recs,key=lambda x: x[1],reverse=True)]
    return final_rec[:num_recom]

In [491]:
USERID=1
num_recom = 20
for item in get_recommends_for_user(USERID,U,VT,num_recom):
    print(item)
    print("Actual Rating: ", mat.toarray()[USERID][item[0]])
    #print(movies[movies.movieId == str(item[0])].Title,'\n')

(2055, 0.001850468730998939)
Actual Rating:  0.0
(1973, 0.001805551770540473)
Actual Rating:  0.0
(4523, 0.0017806898322986027)
Actual Rating:  0.0
(4352, 0.0017782810948929525)
Actual Rating:  0.0
(6548, 0.0017764858483967235)
Actual Rating:  0.0
(2163, 0.0017178366611682412)
Actual Rating:  0.0
(2017, 0.0016368867893164919)
Actual Rating:  0.0
(5215, 0.001626888260165753)
Actual Rating:  0.0
(1635, 0.0015849165675062631)
Actual Rating:  0.0
(135, 0.0015389047105925536)
Actual Rating:  0.0
(423, 0.0015297085761235307)
Actual Rating:  0.0
(3804, 0.0015134545490765009)
Actual Rating:  0.0
(5417, 0.0015090491912795718)
Actual Rating:  0.0
(1626, 0.0015040146748406208)
Actual Rating:  0.0
(4683, 0.0015001699616872008)
Actual Rating:  0.0
(922, 0.0014973840446697183)
Actual Rating:  0.0
(3038, 0.001483801029433646)
Actual Rating:  0.0
(271, 0.0014758026435121947)
Actual Rating:  0.0
(1352, 0.0014599664027197912)
Actual Rating:  0.0
(1852, 0.0014558787346929355)
Actual Rating:  0.0


In [492]:
USERID=73
num_recom = 20
for item in get_recommends_for_user(USERID,U,VT,num_recom):
    #print(item[0])
    find_book_title(item[0])
    find_book_rating(item[0])

Scary Stories to Tell in the Dark
4.05
Corduroy
4.28
Magyk
3.8
ふしぎ遊戯 1
4.08
The Mating
4.02
Manuscrito encontrado em Accra
3.8
Beyond the Highland Mist
3.99
Pawn of Prophecy
4.16
Wings
3.75
The Impossible Knife of Memory
3.93
Under the Banner of Heaven: A Story of Violent Faith
3.98
Mrs. Frisby and the Rats of NIMH
4.13
Du contrat social: ou Principes du droit politique
3.75
Club Dead
4.03
ビューティーポップ 1 [Byūtī poppu 1]
4.15
An Illustrated Life: Drawing Inspiration From The Private Sketchbooks Of Artists, Illustrators And Designers
3.72
NARUTO -ナルト- 巻ノ四十三
4.57
To Tame a Highland Warrior
4.14
Chosen Prey
4.23
Bring Up the Bodies
4.26


In [493]:
find_book_title(1681)

Lady Chatterley's Lover


In [494]:
find_book_rating(1681)

3.49


In [41]:
from scipy.sparse import lil_matrix

mat = lil_matrix((53000,10000)) #linked list matrix

In [42]:
mat.shape

(53000, 10000)

In [ ]:
for u, b, r in zip (user, book, rating):
    mat[u, b]=r

In [ ]:
from scipy.sparse import lil_matrix

In [90]:
mat = lil_matrix((53000,10000)) #linked list matrix

In [72]:
#for i in range(100):
rating_cursor = ratings.aggregate([{'$sample':{'size':5}}]) 
    
for rating in rating_cursor:
        u_list = []
        
        u = float(rating['user_id'])
        u_list.append(u)
        
        b = float(rating['book_id'])
        r = float(rating['rating'])

print(u_list)

   # for u, b, r in zip (u, b, r):
    #    mat[u,b] = r

[1637.0]


In [ ]:
#for i in range(100):
rating_cursor = ratings.aggregate([{'$sample':{'size':5}}]) 
'''    
for rating in rating_cursor:
        u_list = []
        
        u = float(rating['user_id'])
        u_list.append(u)
        
        b = float(rating['book_id'])
        r = float(rating['rating'])

print(u_list)
'''
for rating in rating_cursor:
    
    for u, b, r in zip (float(rating['user_id']), float(rating['book_id']), float(rating['rating'])):
        mat[u,b] = r

In [91]:
rating_cursor = ratings.aggregate([{'$sample':{'size':5}}]) 

In [96]:
rating_cursor #is this lazy?

In [92]:
for rating in rating_cursor:
        u = float(rating['user_id'])
        print(u)
        b = float(rating['book_id'])
        print(b)
        r = float(rating['rating'])
        print(r)
        mat[u, b] = r

11745.0
2979.0
3.0
30118.0
6380.0
4.0
26240.0
2416.0
5.0
4924.0
8402.0
5.0
19777.0
2838.0
2.0


In [83]:
mat.data

array([list([]), list([7.0]), list([8.0]), ..., list([]), list([]),
       list([])], dtype=object)

In [77]:
mat.shape

(53000, 10000)

In [93]:
mat.sum()

19.0

In [95]:
print(mat)

  (4924, 8402)	5.0
  (11745, 2979)	3.0
  (19777, 2838)	2.0
  (26240, 2416)	5.0
  (30118, 6380)	4.0


In [61]:
for u, b, r in zip (u, b, r):
    mat[u,b] = r

In [ ]:
user_list = np.array([])
book_list = np.array([])
rating_list = np.array([])

for rating in rating_cursor:
    user_list = np.append(user_list,(rating['user_id']))
    book_list = np.append(book_list,(float(rating['book_id'])))
    rating_list = np.append(rating_list,(float(rating['rating'])))

In [58]:
int(rating['user_id'])

47524

In [ ]:
mat.shape

In [ ]:
mat.csr #compressed sparse row matrix

In [ ]:
mat.data.nbytes

In [115]:
large_mat = lil_matrix((53425,10001)) #linked list matrix

In [116]:
large_rating_cursor = ratings.find()#aggregate([{'$sample':{'size':100000}}]) 

In [117]:
for rating in large_rating_cursor:
        u = float(rating['user_id'])
        b = float(rating['book_id'])
        r = float(rating['rating'])
        large_mat[u, b] = r

In [118]:
large_mat.data.nbytes

427400

In [111]:
from scipy.sparse import csr_matrix

In [121]:
large_mat.tocsr().data.nbytes

47811832

In [122]:
csr_mat = large_mat.tocsr()

In [124]:
csr_mat.shape

(53425, 10001)

In [164]:
U.shape

(53425, 100)

In [165]:
s.shape

(100,)

In [166]:
VT.shape

(100, 10001)

In [167]:
csr_mat.data

array([5., 4., 5., ..., 5., 4., 4.])

In [171]:
csr_mat.shape

(53425, 10001)

In [260]:
U, s, VT = svds(csr_mat,k=175)

In [261]:
def get_recommendations_for_user(userID, U, VT, num_recom):
    recs = []
    for item in range(VT.T.shape[0]):
        recs.append([item+1,np.dot(U[userID-1],VT.T[item])])
    final_rec = [(i[0],i[1]) for i in sorted(recs,key=lambda x: x[1],reverse=True)]
    return final_rec[:num_recom]

In [262]:
def item_similarity(item1,item2):
    return np.dot(VT.T[item1],VT.T[item2])

In [263]:
USERID=1
num_recom = 10
for item in get_recommendations_for_user(USERID,U,VT,num_recom):
    find_book_title(item[0])
    find_book_rating(item[0])

The Hunger Games
4.34
Harry Potter and the Philosopher's Stone
4.44
Twilight
3.57
To Kill a Mockingbird
4.25
The Great Gatsby
3.89
The Fault in Our Stars
4.26
The Hobbit or There and Back Again
4.25
The Catcher in the Rye
3.79
Angels & Demons
3.85
Pride and Prejudice
4.24


In [265]:
USERID=4000
num_recom = 10
for item in get_recommendations_for_user(USERID,U,VT,num_recom):
#    print(item[0])
    find_book_title(item[0])
    find_book_rating(item[0])    

Le Petit Prince
4.28
Charlotte's Web
4.15
The Casual Vacancy
3.28
A Wrinkle in Time
4.04
Dune
4.19
The Two Towers
4.42
The Fault in Our Stars
4.26
Miss Peregrine’s Home for Peculiar Children
3.89
Gone Girl
4.03
Safe Haven
4.19


In [267]:
USERID=500
num_recom = 10
for item in get_recommendations_for_user(USERID,U,VT,num_recom):
#    print(item[0])
    find_book_title(item[0])
    find_book_rating(item[0])

Gone with the Wind
4.28
The Lion, the Witch and the Wardrobe
4.19
The Time Machine
3.87
The Glass Castle
4.24
The Joy Luck Club
3.9
Water for Elephants
4.07
A Time to Kill
4.03
Drawing on the Right Side of the Brain
3.84
The Notebook
4.06
The Picture of Dorian Gray
4.06


In [269]:
USERID=5500
num_recom = 10
for item in get_recommendations_for_user(USERID,U,VT,num_recom):
#    print(item[0])
    find_book_title(item[0])
    find_book_rating(item[0])

Wuthering Heights
3.82
The Selection
4.15
Three Cups of Tea
3.64
The Hunt for Red October
4.01
A Game of Thrones
4.45
Fahrenheit 451
3.97
The Last Song
4.14
Jane Eyre
4.1
A Thousand Splendid Suns
4.34
Stargirl
3.74


In [272]:
USERID=45000
num_recom = 10
for item in get_recommendations_for_user(USERID,U,VT,num_recom):
#    print(item[0])
    find_book_title(item[0])
    find_book_rating(item[0])

The Outsiders
4.06
Eat, pray, love: one woman's search for everything across Italy, India and Indonesia
3.51
Gulliver’s Travels
3.55
Post-Mortem:  A Mystery Introducing Dr. Kay Scarpetta
4.01
The Pact
4.0
Shantaram
4.26
I'll Give You the Sun
4.14
The Sweetness at the Bottom of the Pie
3.81
Aesopica
4.05
The Dharma Bums
3.94


In [275]:
USERID=35000
num_recom = 10
for item in get_recommendations_for_user(USERID,U,VT,num_recom):
#    print(item[0])
    find_book_title(item[0])
    find_book_rating(item[0])

The Girl on the Train
3.88
The Glass Castle
4.24
Wuthering Heights
3.82
Het Achterhuis: Dagboekbrieven 14 juni 1942 - 1 augustus 1944
4.1
Män som hatar kvinnor
4.11
Catching Fire
4.3
The Curious Incident of the Dog in the Night-Time
3.85
Eat, pray, love: one woman's search for everything across Italy, India and Indonesia
3.51
Harry Potter and the Philosopher's Stone
4.44
Fallen
3.74
